In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
from itertools import combinations
import networkx as nx
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, auc, roc_curve
import missingno as msno
import ast
from collections import Counter
import statsmodels.regression.recursive_ls as rls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pyvis.network import Network

%load_ext autoreload
%autoreload 2

In [8]:
df = pd.read_pickle('data/df_movie.pkl')

In [9]:
df1 = pd.read_pickle('data/df_pairs.pkl')
df1.head()

,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre
0,'Til There Was You,1997.0,"(Alice Drummond, Christine Ebersole)",3525125.0,4.8,Alice Drummond,Christine Ebersole,25.0,3,3.612798e+07,False,False,0,False,None
1,'Til There Was You,1997.0,"(Alice Drummond, Craig Bierko)",3525125.0,4.8,Alice Drummond,Craig Bierko,37.0,5,1.416641e+07,False,False,0,False,None
2,'Til There Was You,1997.0,"(Alice Drummond, Dylan McDermott)",3525125.0,4.8,Alice Drummond,Dylan McDermott,34.0,1,1.315054e+07,False,False,0,False,None
3,'Til There Was You,1997.0,"(Alice Drummond, Jeanne Tripplehorn)",3525125.0,4.8,Alice Drummond,Jeanne Tripplehorn,36.0,2,1.241967e+08,False,False,0,False,None
4,'Til There Was You,1997.0,"(Alice Drummond, Jennifer Aniston)",3525125.0,4.8,Alice Drummond,Jennifer Aniston,41.0,3,4.310507e+07,False,False,0,False,None


In [10]:
columns_inf = ['year', 'amount','inflation rate']
inflation = pd.read_table('data/inflation_data.csv', header=None, names=columns_inf,sep=',')
inflation = inflation.drop(index=0)

#From https://www.officialdata.org/us/inflation/1888?amount=1

value_in_2023 = [32.39,33.44,33.81,33.81,33.81,34.19,35.78,36.63,36.63,37.07,
                   37.07,37.07,36.63,36.20,35.78,34.96,34.57,34.96,34.19,32.73,
                   33.44,33.81,32.39,32.39,31.72,31.08,30.77,30.46,28.23,24.04,
                   20.38,17.78,15.38,17.19,18.31,17.99,17.99,17.58,17.38,17.68,
                   17.99,17.99,18.42,20.24,22.46,23.67,22.96,22.46,22.13,21.37,
                   21.82,22.13,21.98,20.93,18.88,17.78,17.48,17.09,15.78,13.80,
                   12.77,12.93,12.77,11.83,11.61,11.52,11.44,11.48,11.31,10.95,
                   10.65,10.57,10.39,10.29,10.19,10.05,9.92,9.77,9.50,9.21,8.84,
                   8.38,7.93,7.60,7.36,6.93,6.24,5.72,5.41,5.08,4.72,4.24,3.73,
                   3.38,3.19,3.09,2.96,2.86,2.81,2.71,2.60,2.48,2.35,2.26,2.19,
                   2.13,2.08,2.02,1.96,1.92,1.89,1.85,1.79,1.74,1.71,1.67,1.63,
                   1.58,1.53,1.48,1.43,1.43,1.41,1.37,1.34,1.32,1.30,1.30,1.28,
                   1.26,1.22,1.20,1.19,1.14,1.05,1]

inflation["Inflation Factor for 2023"] = value_in_2023
inflation["year"] = inflation["year"].astype(float)

df1['Inflation Factor for 2023'] = df1['Movie_release'].map(inflation.set_index('year')['Inflation Factor for 2023'])
df1['2023 valued revenue'] = df1['Movie_revenue'] * df1['Inflation Factor for 2023']

df1 = df1.sort_values(by=['2023 valued revenue'],ascending = False)
df1.head(5)


,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre,Inflation Factor for 2023,2023 valued revenue
505531,Titanic,1997.0,"(Ewan Stewart, Suzy Amis)",2.185372e+09,7.9,Ewan Stewart,Suzy Amis,5.0,7,1.905096e+09,False,True,0,False,None,1.92,4.195915e+09
505447,Titanic,1997.0,"(Bill Paxton, Kathy Bates)",2.185372e+09,7.9,Bill Paxton,Kathy Bates,7.0,4,3.325277e+07,False,False,0,False,None,1.92,4.195915e+09
505434,Titanic,1997.0,"(Bill Paxton, Billy Zane)",2.185372e+09,7.9,Bill Paxton,Billy Zane,11.0,11,9.950235e+07,False,False,1,False,None,1.92,4.195915e+09
505435,Titanic,1997.0,"(Bill Paxton, Danny Nucci)",2.185372e+09,7.9,Bill Paxton,Danny Nucci,13.0,13,1.491934e+08,False,False,0,False,None,1.92,4.195915e+09
505436,Titanic,1997.0,"(Bill Paxton, David Warner)",2.185372e+09,7.9,Bill Paxton,David Warner,14.0,4,2.243814e+07,False,False,0,True,Thriller,1.92,4.195915e+09


In [24]:
df2 = df1.copy()


#Filter the years to have only the films from 1980 to 1985 first
df2 = df2[(df2['Movie_release'] >= 1980) & (df2['Movie_release'] <= 2020)]


# Step 1: Create a mapping DataFrame for 'Actor_pairs' to 'Actor1', 'Actor2', and 'Genre'
actor_pairs_mapping = df2[['Actor_pairs', 'Actor1', 'Actor2', 'Genre']].drop_duplicates()

# Step 2: Grouping by 'Actor_pairs' and calculating the required metrics along with including 'Genre'
grouped_df = df2.groupby('Actor_pairs').agg(
    Average_Movie_revenue=pd.NamedAgg(column='2023 valued revenue', aggfunc='mean'),
    Average_Movie_rating=pd.NamedAgg(column='Movie_rating', aggfunc='mean'),
    Count=pd.NamedAgg(column='Movie_name', aggfunc='count')
)

# Reset index in the grouped DataFrame
grouped_df.reset_index(inplace=True)

# Step 3: Merge the aggregated DataFrame with the mapping DataFrame
# Note: The merge may result in multiple rows per actor pair if they have multiple genres.
final_df = pd.merge(grouped_df, actor_pairs_mapping, on='Actor_pairs')

final_df

,Actor_pairs,Average_Movie_revenue,Average_Movie_rating,Count,Actor1,Actor2,Genre
0,"(50 Cent, Adewale Akinnuoye-Agbaje)",7.337919e+07,5.4,1,50 Cent,Adewale Akinnuoye-Agbaje,None
1,"(50 Cent, Al Pacino)",1.097485e+08,6.0,1,50 Cent,Al Pacino,None
2,"(50 Cent, Alan Blumenfeld)",1.097485e+08,6.0,1,50 Cent,Alan Blumenfeld,Crime Fiction
3,"(50 Cent, Alan Rosenberg)",1.097485e+08,6.0,1,50 Cent,Alan Rosenberg,None
4,"(50 Cent, Ambyr Childers)",3.619071e+06,5.6,1,50 Cent,Ambyr Childers,Crime Fiction
...,...,...,...,...,...,...,...
474803,"(Zhenwei Wang, Zhiheng Wang)",5.063677e+08,6.2,1,Zhenwei Wang,Zhiheng Wang,Action/Adventure
474804,"(Zoe Saldana, Zulay Henao)",9.952805e+07,6.2,1,Zoe Saldana,Zulay Henao,None
474805,"(Zoe Saldana, Óscar Jaenada)",4.142541e+07,6.2,1,Zoe Saldana,Óscar Jaenada,Thriller
474806,"(Zoe Saldana, Željko Ivanek)",1.540308e+07,8.6,1,Zoe Saldana,Željko Ivanek,Thriller


In [28]:
from sklearn.preprocessing import MinMaxScaler


# Filter to only keep real duos
duos = final_df[final_df['Count'] >=3]

# Creating a copy of the DataFrame slice
duos_standardized = duos.copy()

# Initialize the StandardScaler
standard_scaler = MinMaxScaler()

# Selecting the columns to be normalized
cols_to_normalize = ['Average_Movie_revenue', 'Average_Movie_rating']

# Applying normalization to the selected columns
duos_standardized[cols_to_normalize] = standard_scaler.fit_transform(duos_standardized[cols_to_normalize])

def round_down_to_nearest_05(number):
    return np.floor(number / 0.05) * 0.05

duos_standardized['Average_Movie_revenue'] = duos_standardized['Average_Movie_revenue'].apply(round_down_to_nearest_05)


rating_stand = duos_standardized.sort_values(by=["Average_Movie_rating","Average_Movie_revenue"], ascending= False)


revenue_stand = duos_standardized.copy()
revenue_stand = duos_standardized.sort_values(by=["Average_Movie_revenue","Average_Movie_rating"], ascending= False)


rating_stand.reset_index(drop=True, inplace=True)
rating_stand['rank'] = rating_stand.index + 1   # Adding 1 to start the ranking from 1

revenue_stand.reset_index(drop=True, inplace=True)
revenue_stand['rank'] = revenue_stand.index + 1   # Adding 1 to start the ranking from 1


for i in range(1, len(rating_stand)):
    # Check if the current row has the same speed and mass as the previous row
    if (rating_stand.loc[i, 'Average_Movie_revenue'] == rating_stand.loc[i-1, 'Average_Movie_revenue']) and (rating_stand.loc[i, 'Average_Movie_rating'] == rating_stand.loc[i-1, 'Average_Movie_rating']):
        # Update the rank to be the same as the previous row
        rating_stand.loc[i, 'rank'] = rating_stand.loc[i-1, 'rank']
    
for i in range(1, len(revenue_stand)):
    # Check if the current row has the same speed and mass as the previous row
    if (revenue_stand.loc[i, 'Average_Movie_revenue'] == revenue_stand.loc[i-1, 'Average_Movie_revenue']) and (revenue_stand.loc[i, 'Average_Movie_rating'] == rating_stand.loc[i-1, 'Average_Movie_rating']):
        # Update the rank to be the same as the previous row
        revenue_stand.loc[i, 'rank'] = revenue_stand.loc[i-1, 'rank']

length = len(rating_stand)

rating_stand['rank_ratio']  = (length - (rating_stand['rank']-1))/ length
revenue_stand['rank_ratio']  = (length - (revenue_stand['rank']-1))/ length

# Function to transform x to y and create a tuple
def transform(x):
    if x >= 0.5:
        y = (x - 0.5) * 2  
        return (0, y, 0.3)
    else:
        y = np.abs((x - 0.5) * 2)
        return (y, 0, 0.3)

# Apply the transformation
rating_stand['Color'] = rating_stand['rank_ratio'].apply(transform)
revenue_stand['Color'] = revenue_stand['rank_ratio'].apply(transform)

In [35]:
rating_stand


,Actor_pairs,Average_Movie_revenue,Average_Movie_rating,Count,Actor1,Actor2,Genre,rank,rank_ratio,Color
0,"(Andy Serkis, Billy Boyd)",1.00,1.000000,3,Andy Serkis,Billy Boyd,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
1,"(Andy Serkis, Cate Blanchett)",1.00,1.000000,3,Andy Serkis,Cate Blanchett,None,1,1.000000,"(0, 1.0, 0.3)"
2,"(Andy Serkis, Christopher Lee)",1.00,1.000000,3,Andy Serkis,Christopher Lee,None,1,1.000000,"(0, 1.0, 0.3)"
3,"(Andy Serkis, David Wenham)",1.00,1.000000,3,Andy Serkis,David Wenham,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
4,"(Andy Serkis, Dominic Monaghan)",1.00,1.000000,3,Andy Serkis,Dominic Monaghan,Fantasy Adventure,1,1.000000,"(0, 1.0, 0.3)"
...,...,...,...,...,...,...,...,...,...,...
2172,"(Loretta Devine, Regina Hall)",0.00,0.116906,4,Loretta Devine,Regina Hall,None,2172,0.002756,"(0.994487827285255, 0, 0.3)"
2173,"(Anthony Anderson, Loretta Devine)",0.00,0.115108,3,Anthony Anderson,Loretta Devine,Crime Fiction,2174,0.001837,"(0.99632521819017, 0, 0.3)"
2174,"(David Mann, Tamela Mann)",0.05,0.079137,3,David Mann,Tamela Mann,None,2175,0.001378,"(0.9972439136426274, 0, 0.3)"
2175,"(David Mann, Tyler Perry)",0.05,0.079137,3,David Mann,Tyler Perry,None,2175,0.001378,"(0.9972439136426274, 0, 0.3)"


In [30]:
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

# Function to compute average color
def average_color(colors):
    avg = np.mean(colors, axis=0)
    return rgb_to_hex(avg)


In [36]:
from pyvis.network import Network
from collections import defaultdict


# Function to convert RGB to Hexadecimal
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

# Function to compute average color
def average_color(colors):
    avg = np.mean(colors, axis=0)
    return rgb_to_hex(avg)

# Initialize PyVis Network with white background
net = Network(notebook=True, 
              cdn_resources="remote", 
              bgcolor="white",  # Change to white background
              font_color="black",  # Adjust font color for visibility
              height="500px",
              select_menu=True,
              filter_menu=True)

# Set network options
net.set_options("""
const options = {
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -84,
      "centralGravity": 0.09,
      "springLength": 20,
      "springConstant": 0.035
    },
    "minVelocity": 0.18,
    "solver": "forceAtlas2Based"
  }
}
""")

# Counting edges for each node and storing edge colors
edge_count = defaultdict(int)
edge_colors = defaultdict(list)

for _, row in rating_stand.iterrows():
    edge_count[row['Actor1']] += 1
    edge_count[row['Actor2']] += 1
    color = row['Color']
    edge_colors[row['Actor1']].append(color)
    edge_colors[row['Actor2']].append(color)

# Add nodes to the network with size based on edge count and color based on average edge color
for node, count in edge_count.items():
    avg_color = average_color(edge_colors[node])
    net.add_node(node, size=count*5+1, color=avg_color)  # Adjust size scaling factor as needed

# Add edges to the network with color conversion
for _, row in rating_stand.iterrows():
    color_hex = rgb_to_hex(row['Color'])
    net.add_edge(row['Actor1'], row['Actor2'], value=row['Count']*5, color=color_hex)

# Save or show the network
net.show("actor_networkkk.html")



actor_networkkk.html


In [18]:
from pyvis.network import Network
from collections import defaultdict

# Function to convert RGB to Hexadecimal
def rgb_to_hex(rgb):
    return '#{:02x}{:02x}{:02x}'.format(int(rgb[0]*255), int(rgb[1]*255), int(rgb[2]*255))

# Function to compute average color
def average_color(colors):
    avg = np.mean(colors, axis=0)
    return rgb_to_hex(avg)

# Initialize PyVis Network with white background
net = Network(notebook=True, 
              cdn_resources="remote", 
              bgcolor="white",  # Change to white background
              font_color="black",  # Adjust font color for visibility
              height="500px",
              select_menu=True,
              filter_menu=True)

# Set network options
net.set_options("""
const options = {
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -84,
      "centralGravity": 0.09,
      "springLength": 20,
      "springConstant": 0.035
    },
    "minVelocity": 0.18,
    "solver": "forceAtlas2Based"
  }
}
""")

# Counting edges for each node and storing edge colors
edge_count = defaultdict(int)
edge_colors = defaultdict(list)

for _, row in revenue_stand.iterrows():
    edge_count[row['Actor1']] += 1
    edge_count[row['Actor2']] += 1
    color = row['Color']
    edge_colors[row['Actor1']].append(color)
    edge_colors[row['Actor2']].append(color)

# Add nodes to the network with size based on edge count and color based on average edge color
for node, count in edge_count.items():
    avg_color = average_color(edge_colors[node])
    net.add_node(node, size=count*5+1, color=avg_color)  # Adjust size scaling factor as needed

# Add edges to the network with color conversion
for _, row in revenue_stand.iterrows():
    color_hex = rgb_to_hex(row['Color'])
    net.add_edge(row['Actor1'], row['Actor2'], value=row['Count']*2+3, color=color_hex)

# Save or show the network
net.show("actor_network.html")


actor_network.html


In [31]:
import community as community_louvain
import networkx as nx
from pyvis.network import Network
from collections import defaultdict
import numpy as np
import matplotlib.pyplot as plt

# Create a NetworkX graph from the PyVis network
G = nx.Graph()

# Recompute edge_count and edge_colors, if not already computed
edge_count = defaultdict(int)
edge_colors = defaultdict(list)

for _, row in rating_stand.iterrows():
    G.add_edge(row['Actor1'], row['Actor2'], weight=row['Count']*2+5)
    edge_count[row['Actor1']] += 1
    edge_count[row['Actor2']] += 1
    color = row['Color']
    edge_colors[row['Actor1']].append(color)
    edge_colors[row['Actor2']].append(color)

# Compute the best partition (community detection)
partition = community_louvain.best_partition(G)

# Initialize PyVis Network with the same settings
net = Network(notebook=True, cdn_resources="remote", bgcolor="white", font_color="black", height="500px", select_menu=True, filter_menu=True)

# Add nodes to the network with size based on edge count and color based on community
for node, count in edge_count.items():
    community_id = partition[node]
    # Using matplotlib's tab20 colormap
    net.add_node(node, size=count*5+3, color=rgb_to_hex(plt.cm.tab20(community_id)[:3]))

# Add edges to the network without specific color (since community colors are now used)
for _, row in duos_standardized.iterrows():
    net.add_edge(row['Actor1'], row['Actor2'], value=row['Count'])

# Save or show the network
net.show("actor_network.html")


actor_network.html


In [41]:
import networkx as nx
import community as community_louvain
import numpy as np
from collections import defaultdict

# Create a new NetworkX graph
G = nx.Graph()

# Add nodes and edges from the DataFrame
for _, row in rating_stand.iterrows():
    G.add_edge(row['Actor1'], row['Actor2'], weight=row['Count'])

# Detect communities (clusters) using the Louvain method
partition = community_louvain.best_partition(G, weight='weight')

# Group nodes by their cluster
clusters = defaultdict(list)
for node, cluster_id in partition.items():
    clusters[cluster_id].append(node)

# Filter clusters with more than 5 nodes
large_clusters = {k: v for k, v in clusters.items() if len(v) > 5}

# Analysis for each large cluster
cluster_averages = {}

for cluster_id, nodes in large_clusters.items():
    # Extract rows from DataFrame that belong to the current cluster
    cluster_rows = rating_stand[(rating_stand['Actor1'].isin(nodes)) | (rating_stand['Actor2'].isin(nodes))]

    # Calculate average rank for the cluster
    average_rank = cluster_rows['rank'].mean() if not cluster_rows.empty else None

    # Store the results
    cluster_averages[cluster_id] = {
        'actors': nodes,
        'average_rank': average_rank
    }

# Sort clusters by average rank in ascending order
sorted_cluster_averages = dict(sorted(cluster_averages.items(), key=lambda item: item[1]['average_rank']))

# Output the cluster analysis in sorted order
for cluster_id, info in sorted_cluster_averages.items():
    print(f"Cluster {cluster_id}:")
    print(f"Actors: {info['actors']}")
    print(f"Average Rank: {info['average_rank']}\n")


Cluster 0:
Actors: ['Andy Serkis', 'Billy Boyd', 'Cate Blanchett', 'Christopher Lee', 'David Wenham', 'Dominic Monaghan', 'Elijah Wood', 'Hugo Weaving', 'Ian McKellen', 'John Rhys-Davies', 'Liv Tyler', 'Orlando Bloom', 'Sala Baker', 'Sean Astin', 'Sean Bean', 'Viggo Mortensen', 'Peter Jackson', 'Ian Holm', 'Brad Dourif', 'Hugh Jackman', 'Rebecca Romijn', 'Aaron Stanford', 'Anna Paquin', 'Bruce Davison', 'Famke Janssen', 'Halle Berry', 'James Marsden', 'Patrick Stewart', 'Jacek Koman', 'Richard Roxburgh', 'Brent Spiner', 'Gates McFadden', 'Jonathan Frakes', 'LeVar Burton', 'Marina Sirtis', 'Michael Dorn', 'Danny Mann', 'Miriam Margolyes', 'Magda Szubanski', 'Gretchen Mol']
Average Rank: 386.8952380952381

Cluster 2:
Actors: ['Gary Oldman', 'Alfred Enoch', 'Timothy Spall', 'Bonnie Wright', 'Daniel Radcliffe', 'David Bradley', 'Devon Murray', 'Emma Watson', 'Geraldine Somerville', 'James Phelps', 'Jamie Waylett', 'Joshua Herdman', 'Maggie Smith', 'Mark Williams', 'Matthew David Lewis', 'M